In [ ]:
from irace2 import irace, norm_sample, truncated_poisson, truncated_skellam
import numpy as np
import random
import scipy.stats as stats
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tqdm import tqdm
from sklearn import preprocessing
import pandas as pd
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import statsmodels.stats.weightstats as stats
import scipy.stats as ss
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,cross_val_score
from scipy.stats import norm, poisson, skellam
from copy import copy, deepcopy

## Get Data

In [ ]:
df = pd.read_excel('Impulsividade.xls')

In [ ]:
X = preprocessing.normalize(df.drop(columns=['BIS attentional',
            'BIS motor',
            'BIS nonplanning',
            'BIS total',
            'BIS 2F inhibitory control',	
            'BIS 2F nonplanning']).to_numpy())
y = df['BIS attentional'].to_numpy()

## Models

In [ ]:
#all the parameters being configures must be set beforehand
models = [RandomForestRegressor(n_estimators=100,max_depth=5),
    XGBRegressor(n_estimators=100,max_depth=6)]

parameters_dict = {
    'RandomForestRegressor': {'n_estimators': lambda loc: truncated_skellam(loc, mu1=10, mu2=10, min=1), 
                                'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1)},
    'XGBRegressor': {'sample_type': ['uniform','weighted'], 
                        'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1)}
}

## Run race

In [ ]:
stat_test = ss.ttest_rel #stats.ttest_ind, stats.mannwhitneyu
scoring = 'neg_mean_absolute_error'
cv = 10

pop, pop_scores = irace(models, X, y, lambda x: x > 100, stat_test, parameters_dict, pop_size = 20, cv = cv, scoring=scoring)


In [ ]:
print('RF')
scores = cross_val_score(models[0], X, y, cv=cv, scoring=scoring)    
print(f'{np.mean(scores)} +- {np.std(scores)}')

print('XGB')
scores = cross_val_score(models[1], X, y, cv=cv, scoring=scoring)    
print(f'{np.mean(scores)} +- {np.std(scores)}')

print()
for i in range(len(pop)):
    print(pop[i])
    scores = cross_val_score(pop[i], X, y, cv=cv, scoring=scoring) 
    print(f'{np.mean(scores)} +- {np.std(scores)}')